In [1]:
# для обрабртки текста
!pip install  --upgrade gensim

import gensim
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from scipy.io import mmread
from nltk.tokenize import sent_tokenize
from nltk.tokenize import RegexpTokenizer
#from gensim import keywords
#from gensim import summarize
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
import logging
import multiprocessing
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split







In [2]:
from keras.src.layers.core.embedding import Embedding
from keras import backend as K
from keras.preprocessing.text import Tokenizer
from keras.layers import Input, Dense, concatenate, Activation, Dropout, Conv1D
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers
from keras.models import Model
from keras.layers import GlobalMaxPooling1D
from keras.callbacks import ModelCheckpoint


In [3]:
from keras.preprocessing.text import Tokenizer

In [4]:
path_negative = '/content/negative.csv'
path_positive = '/content/positive.csv'

In [5]:
n = ['id', 'date', 'name', 'text', 'typr', 'rep', 'rtw', 'faw', 'stcount', 'foll', 'frien', 'listcount']
data_positive = pd.read_csv(path_positive, sep =';',
                            error_bad_lines = False,
                            names = n, usecols = ['text'])
data_negative = pd.read_csv(path_negative, sep = ';',
                            error_bad_lines = False,
                            names = n, usecols = ['text'])

<ipython-input-5-d9461aaea155>:2: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_positive = pd.read_csv(path_positive, sep =';',
<ipython-input-5-d9461aaea155>:5: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  data_negative = pd.read_csv(path_negative, sep = ';',


In [6]:
data_positive.head()

,text
0,"@first_timee хоть я и школота, но поверь, у на..."
1,"Да, все-таки он немного похож на него. Но мой ..."
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...
3,"RT @digger2912: ""Кто то в углу сидит и погибае..."
4,@irina_dyshkant Вот что значит страшилка :D\nН...


In [7]:
data_negative.head()

,text
0,на работе был полный пиддес :| и так каждое за...
1,"Коллеги сидят рубятся в Urban terror, а я из-з..."
2,@elina_4post как говорят обещаного три года жд...
3,"Желаю хорошего полёта и удачной посадки,я буду..."
4,"Обновил за каким-то лешим surf, теперь не рабо..."


In [8]:
# формируем сбалансированный датасет
sample_size = min(data_positive.shape[0], data_negative.shape[0])

raw_data = np.concatenate((data_positive['text'].values[:sample_size],
                           data_negative['text'].values[:sample_size]), axis = 0)

# делаем метку Y

labels = [1] * sample_size + [0] * sample_size

In [9]:
# обработка текста
def preprocessing_text(text):
  text = text.lower().replace("ё", "e")
  text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', text)
  text = re.sub('@[^\s]+', 'USER', text)
  text = re.sub(' +', '', text)
  return text.strip()

# формируем x train
data = [preprocessing_text(t) for t in raw_data]

In [10]:
# train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42)

In [14]:
x_train[32]

'USERпочему?\nпростоменямамапопросилазавтракнейнаработуприехать,поделу.\nпоэтомузавтрауменянеполучается('

In [15]:
# подготовка текста
# высота матрицы (максимальное число слов в твитере)
SENTENCE_LENGTH = 50

# размер словаря
num_words = 100000

def get_sequences(tokenizer, x):
  # предложения
  sequences = tokenizer.texts_to_sequences(x)
  return pad_sequences(sequences, maxlen = SENTENCE_LENGTH )

# создаем и обучаем токенайзер
tokenizer = Tokenizer(num_words)
tokenizer.fit_on_texts(x_train)

# отображаем текст в массив idx токенов
x_train_seq =  get_sequences(tokenizer, x_train)
x_train_seq =  get_sequences(tokenizer, x_test)

In [16]:
# w2v
logging.basicConfig(format = '%(asctime)s : %(levelname)s', level = logging.INFO)
# обучаем модель
model = Word2Vec(data,  window = 5, min_count = 3, workers = multiprocessing.cpu_count())
model.save('modeltweet.w2v')

In [17]:
model.wv.most_similar('@')

[('✨', 0.582039475440979),
 (']', 0.5434931516647339),
 ('\xad', 0.5419917702674866),
 ('ı', 0.5341613292694092),
 ('형', 0.5120162963867188),
 ('주', 0.5091188549995422),
 ('❗', 0.5002294778823853),
 ('{', 0.49667951464653015),
 ('저', 0.49386751651763916),
 ('=', 0.48908260464668274)]

In [18]:
# word2Vec
w2v_model = Word2Vec.load('modeltweet.w2v')
DIM = w2v_model.vector_size

# интциализируем матрицу embedding слоя нулями
embedding_matrix = np.zeros((num_words, DIM))

# добавляем словарь в embadding

for word, i in tokenizer.word_index.items():
  if i >= num_words:
    break
  if word in w2v_model.wv.vocab.keys():
    embedding_matrix[i] = w2v_model.wv[word]


AttributeError: ignored

In [19]:
# строим нейронную сеть
tweet_input = Input(shape = (SENTENCE_LENGTH,), dtype = 'int32')
tweet_encoder = Embedding(num_words, DIM, input_length=SENTENCE_LENGTH,
                          weights = [embedding_matrix], trainable = False)(tweet_input)

In [21]:
from nltk import accuracy
# нейронная сеть
branches = []

x = Dropout(0.2)(tweet_encoder)

for size, filter_count in [(2, 10), (3, 10), (4, 10), (5, 10)]:
  for i in range(filter_count):
    branch = Conv1D(filters=1, kernel_size = size,
                    padding = 'valid',
                    activation = 'relu')(x)
    branch = GlobalMaxPooling1D()(branch)
    branches.append(branch)

x = concatenate(branches, axis = 1)
x = Dropout(0.2)(x)
x = Dense(30, activation='relu')(x)
x = Dense(1)(x)
output = Activation('sigmoid')(x)

model = Model(inputs = [tweet_input], outputs = [output])
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 50)]                 0         []                            
                                                                                                  
 embedding (Embedding)       (None, 50, 100)              1000000   ['input_1[0][0]']             
                                                          0                                       
                                                                                                  
 dropout_2 (Dropout)         (None, 50, 100)              0         ['embedding[0][0]']           
                                                                                                  
 conv1d_40 (Conv1D)          (None, 49, 1)                201       ['dropout_2[0][0]']     

In [23]:
# обучение
history = model.fit(np.array(x_train_seq), np.array(y_train),
                    batch_size = 32,
                    epochs = 10,
                    validation_split =0.25,
                    verbose =1)

Epoch 1/10
1050/1050 [==============================] - 67s 54ms/step - loss: 0.6932 - accuracy: 0.5011 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 2/10
1050/1050 [==============================] - 56s 54ms/step - loss: 0.6931 - accuracy: 0.5037 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 3/10
1050/1050 [==============================] - 56s 53ms/step - loss: 0.6932 - accuracy: 0.5030 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 4/10
1050/1050 [==============================] - 57s 54ms/step - loss: 0.6932 - accuracy: 0.5016 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 5/10
1050/1050 [==============================] - 56s 53ms/step - loss: 0.6932 - accuracy: 0.5028 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 6/10
1050/1050 [==============================] - 55s 53ms/step - loss: 0.6931 - accuracy: 0.5025 - val_loss: 0.6932 - val_accuracy: 0.4997
Epoch 7/10
1050/1050 [==============================] - 56s 53ms/step - loss: 0.6932 - accuracy: 0.5028 - val_loss: 0.6932 -